Imports and secrets

In [7]:
import csv
import requests
from dotenv import load_dotenv
import os
from datetime import date

load_dotenv()
NOTION_API = os.getenv('NOTION_API')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

Collect entries from Notion DB

In [8]:
data = []

headers = {
    'Authorization': f"Bearer {NOTION_API}",
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

not_done = True
start_cursor = ''
while (not_done):
    body = {}
    if start_cursor != '':
        body = {
            'start_cursor': start_cursor
        }

    response = requests.post(f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID}/query", headers=headers, json=body)
    if response.status_code != 404:
        json = response.json()

        for row in json['results']:
            props = row['properties']
            name = props['Name']['title'][0]['plain_text']
            height = props['Height (cm)']['number']
            categories = f"[{','.join(list(map(lambda x: x['name'], props['Tags']['multi_select'])))}]"
            sources = f"[{','.join(list(map(lambda x: x['name'], props['Source']['multi_select'])))}]"

            data.append([name, height, categories, sources])

        if json['has_more']:
            start_cursor = json['next_cursor']
        else:
            not_done = False


Save entries to csv

In [9]:
with open(f"data/notion-data-{date.today()}.csv", 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(['Name', 'Height', 'Category', 'Source'])   
    writer.writerows(data)